### Download data

- Lower file size from 17GB to 1.5GB
    - Remove some unnecessary data
    - Compress as xz 

In [1]:
# !wget https://ftp.ebi.ac.uk/pub/databases/GO/goa/UNIPROT/goa_uniprot_all.gaf.gz  -O goa_uniprot_all_ebi.gaf.gz.download
# !mv goa_uniprot_all_ebi.gaf.gz.download goa_uniprot_all_ebi.gaf.gz
# !gunzip -c goa_uniprot_all_ebi.gaf.gz | awk 'BEGIN {OFS="\t";FS="\t"} ($1 == "UniProtKB") {print $2,$4,$5,$7,$9,$14}' | sort -u | xz -T0 > goa_uniprot_all_ebi_filtered.tsv.xz.tmp
# !mv goa_uniprot_all_ebi_filtered.tsv.xz.tmp ../data/raw/gene_ontology/goa_uniprot_all_ebi_filtered.tsv.xz

### Read into pandas

Using optimized dtypes to save space

In [45]:
import pandas as pd
from subpred.util import save_df
DATASET_FOLDER = "../data/datasets"

In [43]:
df_go_ebi = pd.read_table(
    "../data/raw/gene_ontology/goa_uniprot_all_ebi_filtered.tsv.xz",
    # force_update=True,
    header=None,
    names=["Uniprot", "qualifier", "go_id", "evidence_code", "aspect", "date"],
    dtype={
        "Uniprot": "string",
        "qualifier": "category",
        "go_id": "string",
        "evidence_code": "category",
        "aspect": "category",
        "date": "string",
    },
    parse_dates=["date"],
)


### Save pickle containing entire dataset:

TODO which container format? Decide based on test:

In [46]:
save_df(df_go_ebi, "go_all", DATASET_FOLDER)

Filtered dataframe with no IEA terms

In [53]:
df_go_ebi_mf_enables = df_go_ebi[
    (df_go_ebi.qualifier == "enables")&
    (df_go_ebi.aspect == "F")
].drop("date", axis=1)

In [ ]:
save_df(df_go_ebi_mf_enables, "go_mf", DATASET_FOLDER)

In [ ]:
!du -hs ../data/datasets/*

du: cannot access '../data/datasets/*': No such file or directory


In [12]:
# df_go_ebi_filtered = df_go_ebi[df_go_ebi.Uniprot.isin(df_uniprot.index)]
# df_go_ebi_filtered = df_go_ebi_filtered[df_go_ebi_filtered.qualifier == "enables"]
# df_go_ebi_filtered = df_go_ebi_filtered[df_go_ebi_filtered.evidence_code != "IEA"]
# df_go_ebi_filtered = df_go_ebi_filtered[df_go_ebi_filtered.aspect == "F"]
# df_go_ebi_filtered = df_go_ebi_filtered.drop(["qualifier", "aspect", "date"], axis=1).reset_index(drop=True)